In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import HumanMessagePromptTemplate, ChatPromptTemplate
from pydantic import BaseModel, Field
load_dotenv()
import warnings
warnings.filterwarnings(action="ignore")

In [2]:
os.environ["REQUESTS_CA_BUNDLE"] = r"../../ca-bundle-full.crt"

In [3]:
llm = AzureChatOpenAI(
    deployment_name=os.environ["AZURE_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_type=os.environ["OPENAI_API_TYPE"],
) 

In [11]:
class Address(BaseModel):
    loc_str_1: str = Field(description="Location string 1")
    loc_str_2: str = Field(description="Location string 2")
    city: str = Field(description="City of the address")
    state: str = Field(description="State of the address")
    country: str = Field(description="ISO Country of the address")
    is_hq: bool = Field(description="Is this address a Global Headquarters")
class Addresses(BaseModel):
    addresses:list[Address] = Field(description="List of address")

In [12]:
output_parser = PydanticOutputParser(pydantic_object = Addresses)
format_instructions = output_parser.get_format_instructions()

In [13]:
format_instructions

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"Address": {"properties": {"loc_str_1": {"description": "Location string 1", "title": "Loc Str 1", "type": "string"}, "loc_str_2": {"description": "Location string 2", "title": "Loc Str 2", "type": "string"}, "city": {"description": "City of the address", "title": "City", "type": "string"}, "state": {"description": "State of the address", "title": "State", "type": "string"}, "country": {"description": "ISO Country of the address", "title": "Country", "type": "string"}, "is_hq": {"description": "Is this address a Global Headq

In [18]:
human_text = "{instruction}\n{format_instructions}"
message = HumanMessagePromptTemplate.from_template(human_text)
prompt = ChatPromptTemplate.from_messages([message])

chain = prompt | llm | output_parser
chain.invoke({"instruction":"Get list of address of IBM","format_instructions":format_instructions})

Addresses(addresses=[Address(loc_str_1='123 Main St', loc_str_2='', city='New York', state='NY', country='USA', is_hq=True), Address(loc_str_1='456 Park Ave', loc_str_2='Suite 789', city='San Francisco', state='CA', country='USA', is_hq=False)])